In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import glob
import os
import json
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
%aimport utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [32]:
pretrained_model = VGG19(weights='imagenet')

In [33]:
model1 = Model(pretrained_model.inputs, [pretrained_model.get_layer('fc1').output])
model2 = Model(pretrained_model.inputs, [pretrained_model.get_layer('fc2').output])

In [3]:
image_files = glob.glob("/mnt/workspace/Ugallery/images/*.jpg")

In [40]:
id2url_map = utils.get_id2url_map()

In [4]:
artwork_ids = [int(f[f.rfind('/')+1:-4]) for f in image_files]

In [36]:
N = len(artwork_ids)
N

13297

In [37]:
fc1_featmat = np.empty(shape=(N,4096))
fc2_featmat = np.empty(shape=(N,4096))

In [50]:
i_start = 0
i_end = N
utils.process_image_batch(
    [model1, model2], [fc1_featmat, fc2_featmat], image_files, artwork_ids, id2url_map,
    i_start, i_end, preprocess_input, image_target_size=(224, 224))

In [53]:
fc1_featmat.mean(), fc2_featmat.mean()

(1.4553600866371263, 0.48915546352972633)

In [5]:
dir_path = "/mnt/workspace/Ugallery/VGG19/"
os.makedirs(dir_path, exist_ok=True)
fc1_featmat.dump(dir_path + "fc1.npy")
fc2_featmat.dump(dir_path + "fc2.npy")
with open(dir_path + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)